In [12]:
def calc_lora_weight(rank):
    num_hidden_layers=61
    first_k_dense_replace = 3
    num_moe_layer = num_hidden_layers - first_k_dense_replace
    
    hidden_size=7168
    intermediate_size=18432
    moe_intermediate_size=2048
    
    n_routed_experts=256
    num_heads  = 128
    q_lora_rank = 1536
    qk_nope_head_dim = 128
    qk_rope_head_dim = 64
    kv_lora_rank = 512
    v_head_dim=128
    
    n_shared_experts=1
    
    vocab_size = 129280
    
    gate_size = n_routed_experts * hidden_size

    moe_mlp_weights = 3 * 2 * rank * (hidden_size + moe_intermediate_size)
    moe_total_weight = n_routed_experts * moe_mlp_weights
    total_expert_weight = moe_total_weight * num_moe_layer
    return int(round(total_expert_weight/1e9,0))

def calc_num_parameters(
    n_routed_experts,
    rank=8
):
    num_hidden_layers=61
    first_k_dense_replace = 3
    num_moe_layer = num_hidden_layers - first_k_dense_replace
    
    hidden_size=7168
    intermediate_size=18432
    moe_intermediate_size=2048
    
    
    num_heads  = 128
    q_lora_rank = 1536
    qk_nope_head_dim = 128
    qk_rope_head_dim = 64
    kv_lora_rank = 512
    v_head_dim=128
    
    n_shared_experts=1
    
    vocab_size = 129280
    
    gate_size = n_routed_experts * hidden_size
    
    mlp_weights = 3 * hidden_size * intermediate_size
    moe_mlp_weights = 3 * hidden_size * moe_intermediate_size
    
    moe_total_weight = n_routed_experts * moe_mlp_weights
    moe_active_weight = num_experts_per_tok * moe_mlp_weights
    
    q_head_dim = qk_nope_head_dim + qk_rope_head_dim
    q_a_proj = hidden_size * q_lora_rank + q_lora_rank * q_head_dim
    kv_a_proj_with_mqa = hidden_size * (kv_lora_rank  + qk_rope_head_dim) + kv_lora_rank * (num_heads * (q_head_dim - qk_rope_head_dim  + v_head_dim))
    o_proj_weight = num_heads * v_head_dim * hidden_size
    attention_weight = q_a_proj + 2 * kv_a_proj_with_mqa + o_proj_weight
    
    base_weight_per_moe_layer = attention_weight + n_shared_experts * moe_mlp_weights + gate_size
    base_weight_per_mlp_layer = attention_weight + mlp_weights
    
    base_model_weight = base_weight_per_moe_layer * num_moe_layer + base_weight_per_mlp_layer * first_k_dense_replace + 2 * vocab_size * hidden_size
    
    total_expert_weight = moe_total_weight * num_moe_layer
    active_expert_weight = moe_active_weight * num_moe_layer
    
    active_model_weight = active_expert_weight + num_moe_layer + base_model_weight
    total_model_weight = total_expert_weight + num_moe_layer + base_model_weight

    fused_lora_weight = calc_lora_weight(rank)
    
    print(f"{n_routed_experts} => {int(round(total_model_weight/1e9,0))}B @ {fused_lora_weight}B parameters")

In [13]:
calc_lora_weight(8)

7

In [16]:
p = [
    (8,4),
    (8,8),
    (8,16),
    (4,4),
    (4,8),
    (4,16),
    (2,4),
    (2,8),
    (2,16),
]

for elt in p:
    calc_num_parameters(*elt)

8 => 36B @ 3B parameters
8 => 36B @ 7B parameters
8 => 36B @ 13B parameters
4 => 26B @ 3B parameters
4 => 26B @ 7B parameters
4 => 26B @ 13B parameters
2 => 21B @ 3B parameters
2 => 21B @ 7B parameters
2 => 21B @ 13B parameters
